# Simple Zero Shot Classification Example
We are going to automatically classify a support `Ticket` with a `Category`.

[What is Zero Shot classification?](https://weaviate.io/developers/weaviate/api/rest/classification#zero-shot-classification)

First, let's spin up both Weaviate and a transformers-inference model inference


> docker compose up -d


In [1]:
# this will be our data, a Ticket with a body property and some Categories
# we will use the id as a base to generate our UUID
tickets = [
    {"body": "I cannot connect to the internet", "id":1},
    {"body": "I wat to put some text in a paper", "id":2},
    {"body": "My computer is slow, I cannot turn it on", "id":3},
    {"body": "I want to create some spreadsheets, but I cannot open the program", "id":4},
]
# each ticket will have one category
categories = [
    "Network",
    "Printing",
    "Hardware",
    "Software",
]

In [2]:
# Defining both our classes
category_class = {
            "class": 'Category',
            "description": 'support ticket',  
            "properties": [
                {
                "dataType": [ 'text'],
                "description": 'name of category',
                "name": 'name',
                }            
            ]
        }

# We do a cross reference, so Ticket has a property category that references
# the data type ["Category"] we have just defined.

ticket_class = {
            "class": 'Ticket',
            "description": 'support ticket',  
            "properties": [
                {
                "name": 'body',
                "description": 'ticket text',
                "dataType": [ 'text'],
                },
                {
                "name": 'ticket_id',
                "description": 'ticket id',
                "dataType": [ 'number'],
                },                
                {
                "name": 'category',
                "description": 'ticket topic',
                "dataType": ["Category"],
                }
            ]
        }

In [3]:
# lets get our client
import weaviate
client = weaviate.Client("http://localhost:8080")
# clear our classes
client.schema.delete_all()
# and create them before hand
# this is important, as we want to make sure the category property exists
if not client.schema.exists("Category"):
    client.schema.create_class(category_class)
if not client.schema.exists("Ticket"):
    client.schema.create_class(ticket_class)

In [4]:
# now we import our data
print("#### ADDING CATEGORIES")
client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:
    for category in categories:
        cat_id = batch.add_data_object({"name": category}, "Category", weaviate.util.generate_uuid5(category))
        print(category, cat_id)

print("#### ADDING TICKETS")
client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:
    for ticket in tickets:
        ticket_id = batch.add_data_object(
            {"body": ticket["body"], "ticket_id": ticket["id"], "category": None}, "Ticket", 
            weaviate.util.generate_uuid5(ticket["id"])
        )
        print(ticket, ticket_id)

#### ADDING CATEGORIES
Network 52eebe84-8111-55fd-9410-2972b6d8c159
Printing 424ffc16-e702-5e33-b9c9-41d93c973641
Hardware b658ce0d-fc00-5ffe-976d-fc2141a6d2c1
Software b281d52e-8aec-5d7f-b842-2cff8dfed20e
#### ADDING TICKETS
{'body': 'I cannot connect to the internet', 'id': 1} b04965e6-a9bb-591f-8f8a-1adcb2c8dc39
{'body': 'I wat to put some text in a paper', 'id': 2} 4b166dbe-d99d-5091-abdd-95b83330ed3a
{'body': 'My computer is slow, I cannot turn it on', 'id': 3} 98123fde-012f-5ff3-8b50-881449dac91a
{'body': 'I want to create some spreadsheets, but I cannot open the program', 'id': 4} 6ed955c6-506a-5343-9be4-2c0afae02eef


## MAKE SURE TO WAIT DATA TO BE INGESTED AND VECTORIZED!!

In [5]:
# now the fun part
client.classification.schedule()\
            .with_type("zeroshot")\
            .with_class_name("Ticket")\
            .with_classify_properties(["category"])\
            .with_based_on_properties(["body"])\
            .do()

{'basedOnProperties': ['body'],
 'class': 'Ticket',
 'classifyProperties': ['category'],
 'id': '72c650cd-b016-4ae4-9703-5d6ea092f1a4',
 'meta': {'completed': '0001-01-01T00:00:00.000Z',
  'started': '2023-10-24T00:17:42.093Z'},
 'status': 'running',
 'type': 'zeroshot'}

In [6]:
# just like that, you have your items categorized
results = client.query.get("Ticket", "body category{ ... on Category{name}}").do()
for ticket in results["data"]["Get"]["Ticket"]:
    print("#" * 10)
    print("Ticket:", ticket["body"])
    print("Category: ", ticket["category"][0]["name"])

##########
Ticket: I wat to put some text in a paper
Category:  Printing
##########
Ticket: I want to create some spreadsheets, but I cannot open the program
Category:  Software
##########
Ticket: My computer is slow, I cannot turn it on
Category:  Hardware
##########
Ticket: I cannot connect to the internet
Category:  Network
